In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os.path

import keras
from keras.utils import np_utils
from keras import utils, callbacks
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

from sklearn.metrics import accuracy_score, f1_score

import helper

print(f'tensorflow version is {tf.__version__}')


In [ ]:
# Params

classes = 29
batch = 256
epochs = 50
lnr = 0.0001

train_filepath = r'asl_alphabet_train/asl_alphabet_train/'

In [ ]:
(train_img_paths, train_labels), (test_img_paths, test_img_labels) = helper.load_data()

In [ ]:
print(f'Number of pictures: {train_labels.shape[0]}\n')

print(f'Number of different labels: {len(train_labels.unique())}\n')

print(f'Labels: {train_labels.unique()}')

In [ ]:
# Display 30 picture of the dataset with their labels

fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_img_paths[i]))
    ax.set_title(train_labels[i])
plt.tight_layout(pad=.5)
plt.show()


In [ ]:
train_labels.unique()

In [ ]:
# Build Image Data Generator for training
# target size set to default (64,64)

gen = ImageDataGenerator(rescale=1./255, 
                         rotation_range=30,
                         shear_range=10, 
                         validation_split=0.1) 

train = gen.flow_from_directory(train_filepath,
                                target_size=(256,256),
                                color_mode='rgb',
                                class_mode='categorical',
                                batch_size=32,
                                shuffle=True,
                                seed=42,
                                subset="training")

val = gen.flow_from_directory(train_filepath, 
                              target_size=(256,256),
                              color_mode='rgb',
                              class_mode='categorical',
                              batch_size=32,
                              shuffle=True,
                              seed=42,
                              subset="validation")


In [ ]:
# Build a model around VGG_16

adam = Adam(learning_rate=lnr)

model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
model_vgg.trainable = False

model = Sequential([
    keras.Input(shape=(256, 256, 3)),
    model_vgg,
    Flatten(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(29, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'] )

In [ ]:
# Build callbacks to stop training if model worsens for 5 conscutive episodes

earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", 
                                        patience=5, restore_best_weights = True)

history = model.fit(train, validation_data = val, epochs=epochs, shuffle=True, verbose=1, 
                    callbacks = [earlystopping])
